## Example 5: Flows


Flows are containers that maintain context and state across multiple related tasks. They're useful when:
- Multiple Tasks Need Context: Tasks need to share information or build upon each other's results
- Conversation History Matters: When previous interactions influence future responses  
- Complex Workflows: When orchestrating multiple agents or tasks in a specific sequence


The following example demonstrates how to use flows to plan a trip passing the max budget and duration to take into consideration, and then optimizing the trip plan to fit within those constraints.

In [1]:
import controlflow as cf
from typing import List, Dict
from pydantic import BaseModel


In [2]:
# Define our data models
class Destination(BaseModel):
    city: str
    country: str
    activities: List[str]

class TripPlan(BaseModel):
    destinations: List[Destination]
    duration_days: int
    budget_estimate: float

In [4]:
# set parameters
MAX_BUDGET = 1000.0
DURATION_DAYS = 7

In [5]:
# Create specialized agents
travel_advisor = cf.Agent(
    name="Travel Advisor",
    description="Expert in travel planning and recommendations",
    instructions="Help users plan their ideal trip with detailed recommendations."
)

budget_analyst = cf.Agent(
    name="Budget Analyst",
    description="Expert in travel costs and budgeting",
    instructions="Provide realistic budget estimates for travel plans."
)

# Create the main flow with all parameters
@cf.flow(context_kwargs=["budget_limit", "duration_days", "destinations"])
def plan_trip(
    destinations: List[str], 
    budget_limit: float,
    duration_days: int
) -> TripPlan:
    """
    Plan a comprehensive trip including destinations, activities, and budget.
    """
    all_destinations = []
    total_budget = 0.0

    # Process each destination
    for destination in destinations:
        # Get destination details
        destination_details = cf.run(
            f"Analyze {destination} as a travel destination and suggest activities for {duration_days} days",
            agents=[travel_advisor],
            result_type=Destination
        )

        # Get budget estimate
        budget_estimate = cf.run(
            f"Create a budget estimate for {duration_days} days of activities",
            agents=[budget_analyst],
            result_type=float,
            context={"destination": destination_details}
        )

        # Optimize if needed
        if budget_estimate > budget_limit / len(destinations):
            with cf.Flow(name="Budget Optimization") as budget_flow:
                optimized_plan = cf.run(
                    "Optimize the travel plan to fit within budget",
                    agents=[travel_advisor, budget_analyst],
                    context={
                        "original_plan": destination_details,
                        "budget_limit": budget_limit / len(destinations),
                        "duration": duration_days
                    },
                    result_type=Destination
                )
                destination_details = optimized_plan
                budget_estimate = budget_limit / len(destinations)

        all_destinations.append(destination_details)
        total_budget += budget_estimate

    # Create final trip plan
    return TripPlan(
        destinations=all_destinations,
        duration_days=duration_days,
        budget_estimate=total_budget
    )

# Example usage
if __name__ == "__main__":
    # Run the flow with multiple destinations
    trip = plan_trip(
        destinations=["Paris", "London"],
        budget_limit=MAX_BUDGET,
        duration_days=DURATION_DAYS
    )
    
    print("\nTrip Plan:")
    for dest in trip.destinations:
        print(f"\nDestination: {dest.city}, {dest.country}")
        print(f"Activities: {', '.join(dest.activities)}")
    print(f"\nTotal Duration: {trip.duration_days} days")
    print(f"Total Budget Estimate: ${trip.budget_estimate:,.2f}")

Output()

Output()

Output()

Output()

Output()

Output()


Trip Plan:

Destination: Paris, France
Activities: Day 1: Explore the iconic Eiffel Tower and take a Seine River Cruise. (Cost: $50), Day 2: Visit the Louvre Museum and stroll through the Tuileries Garden. (Cost: $20), Day 3: Discover the charm of Montmartre and visit the Sacré-Cœur Basilica. (Cost: $0), Day 4: Take a day trip to the Palace of Versailles. (Cost: $60), Day 5: Explore the Latin Quarter and visit the Panthéon. (Cost: $10), Day 6: Enjoy a shopping day at Champs-Élysées and visit the Arc de Triomphe. (Cost: $63), Day 7: Relax in the Luxembourg Gardens and explore the Musée d'Orsay. (Cost: $16)

Destination: London, United Kingdom
Activities: Day 1: Visit the British Museum and explore Covent Garden., Day 2: Explore Tower Bridge and enjoy a walk around the Tower of London., Day 3: Walk around Westminster Abbey and enjoy St James's Park., Day 4: Visit Trafalgar Square and explore the National Gallery., Day 5: Spend a day in Camden Market and visit Regent's Park., Day 6: Enjo

04:24:58.634 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 552.79551 seconds to run, which is longer than its loop interval of 5.0 seconds.
04:24:58.637 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 552.799704 seconds to run, which is longer than its loop interval of 5.0 seconds.
07:15:15.574 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 904.781009 seconds to run, which is longer than its loop interval of 5.0 seconds.
07:15:15.576 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 904.786621 seconds to run, which is longer than its loop interval of 5 seconds.
07:15:15.578 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 904.786608 seconds to run, which is longer than its loop interval of 5.0 seconds.
09:19:13.640 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 938.845844 seconds to run